In [1]:
OutWorkspace = r"E:\projects\BP\Indicators\indicators.gdb"
sr= arcpy.SpatialReference(5070)

# define rasters used for cell size, extent, and snapping
SEraster= r"E:\projects\BP\1_Extent\BaseBlueprintExtent2022.tif"
SEpoly = r"E:\projects\BP\1_Extent\BPExtent.gdb\BaseBPExtent_2022"

# define inputs
landcover = r"E:\generalGIS\NLCD\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
urbanextent = r"E:\generalgis\TIGER\tl_2017_us_uac10\tl_2017_us_uac10_SE5070.shp"
PADUS = r"E:\generalgis\PAD_US2_1_GDB\PAD_US2_1.gdb\PADUS2_1Combined_Fee_Easement"


In [4]:
#import the reclassed vector from QGIS back in
imppolygon= r"E:\projects\BP\Indicators\UPSfinsel_v2.shp"

In [5]:
#### convert polygon to raster
with arcpy.EnvManager(extent = SEraster, outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    imppolygon2 =  arcpy.conversion.PolygonToRaster(imppolygon, "areaclass", r"E:\projects\BP\Indicators\UrbanParkSizeZND.tif", "CELL_CENTER", "NONE", 30, "BUILD")

In [6]:
## reclassify to merge class zero parks with entire SE BP extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    out_raster_znd = arcpy.sa.Reclassify(imppolygon2, "Value", "0 0;1 1;2 2;3 3;4 4;5 5;NODATA 0", "DATA")

In [7]:
## reclassify marine to nodata - if NLCD = 0 then no data
out_raster_znd2 = arcpy.ia.SetNull("nlcd_2019_land_cover_l48_20210604.img", out_raster_znd, "Value = 0"); out_raster_znd.save(r"E:\projects\BP\Indicators\UrbanParkSizeZND_v2.tif")

In [7]:
# edit RAT 
arcpy.management.AddField(out_raster_znd2, "descript", "text", 25)
arcpy.management.AddField(out_raster_znd2, "Red", "long", 9)
arcpy.management.AddField(out_raster_znd2, "Green", "long", 9)
arcpy.management.AddField(out_raster_znd2, "Blue", "long", 9)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000012: Red already exists
Failed to execute (AddField).


In [8]:
# set code block for next step
codeblock = """
def Reclass(Value):
    if Value == 5:
        return "5 = 75+ acre urban park"
    elif Value == 4:
        return "4 = 50 to <75 acre urban park"
    elif Value == 3:
        return "3 = 30 to <50 acre urban park"
    elif Value == 2:
        return "2 = 10 to <30 acre urban park"
    elif Value == 1:
        return "1 = 5 to <10 acre urban park"
    elif Value == 0:
        return "0 = <5 acre urban park or not identified as an urban park"
"""

In [9]:
# Value R G B

# set code block for next step
codeblock_red = """
def Reclass(Value):
    if Value == 5:
        return "53"
    elif Value == 4:
        return "109"
    elif Value == 3:
        return "165"
    elif Value == 2:
        return "208"
    elif Value == 1:
        return "232"
    elif Value == 0:
        return "255"
"""
codeblock_green = """
def Reclass(Value):
    if Value == 5:
        return "2"
    elif Value == 4:
        return "35"
    elif Value == 3:
        return "81"
    elif Value == 2:
        return "122"
    elif Value == 1:
        return "188"
    elif Value == 0:
        return "255"
"""
codeblock_blue = """
def Reclass(Value):
    if Value == 5:
        return "66"
    elif Value == 4:
        return "115"
    elif Value == 3:
        return "153"
    elif Value == 2:
        return "182"
    elif Value == 1:
        return "197"
    elif Value == 0:
        return "255"
"""

In [10]:
arcpy.management.CalculateField(out_raster_znd2, "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")
arcpy.management.CalculateField(out_raster_znd2, "Red", "Reclass(!Value!)", "PYTHON3", codeblock_red, "LONG")
arcpy.management.CalculateField(out_raster_znd2, "Green", "Reclass(!Value!)", "PYTHON3", codeblock_green, "LONG")
arcpy.management.CalculateField(out_raster_znd2, "Blue", "Reclass(!Value!)", "PYTHON3", codeblock_blue, "LONG")

<Result 'E:\\projects\\BP\\Indicators\\UrbanParkSize_ZND\\UrbanParkSize.tif'>

In [12]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster_znd2 = out_raster_znd2.save(r"E:\projects\BP\Indicators\UrbanParkSize_znd_v6.tif")